In [10]:
import os
from time import time
from tqdm import tqdm
import numpy as np
from PIL import Image
import torch
from torch.nn import Linear, CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
import torchvision.models as models
from torchvision.transforms import transforms
from torchvision.models import ResNet18_Weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if your computer has cuda (gpu) it will use it if not it will use gpu

In [3]:
#Create transformer. Will fit data to enter resnet model

tfm = transforms.Compose([
    transforms.Resize((224, 224)), #standard size of images for resnet
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees = 10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #normalize pictures to fit imagenet format
])

#Create dataset into image folder

TRAIN_ROOT = "/Users/austinjohnson/Downloads/Breast Cancer/train"
TEST_ROOT = "/Users/austinjohnson/Downloads/Breast Cancer/test"

train_ds = ImageFolder(TRAIN_ROOT, transform=tfm)
test_ds = ImageFolder(TEST_ROOT, transform=tfm)

LEN_TRAIN = len(train_ds) #925
LEN_TEST = len(test_ds) #661
print(LEN_TRAIN, LEN_TEST)


print(train_ds.class_to_idx) #maps IDC to 0 and Non_IDC to 1

925 661
{'IDC': 0, 'Non-IDC': 1}


In [4]:
# Data Loader
train_loader = DataLoader(train_ds, batch_size = 30, shuffle = True)
test_loader = DataLoader(test_ds, batch_size = 30, shuffle = True)
#better computers can handle higher batch sizes
#lower batch size the less strain on gpu/cpu
#higher batch sizes do create a higher accuracy though

In [5]:
#Build Model!!

model = models.resnet18(weights = ResNet18_Weights.DEFAULT)
#replace the final (fully connected) layer with what we want
#input features remain the same
model.fc = Linear(in_features = 512, out_features = 2) #we have 2 outputs

model = model.to(device) # move model to gpu

#Optimizer
optimizer = Adam(model.parameters(), lr=3e-4, weight_decay=0.001)
#lr = learning rate 3e-4 is best learning rate for adam
#weight decay helps prevent model overfitting

#Loss Function
loss_fn = CrossEntropyLoss()

In [6]:
# Train Model (3 epochs)
for epoch in range(3):
    start = time()
    tr_acc = 0
    test_acc = 0

    model.train()

    #progress bar
    with tqdm(train_loader, unit="batch") as tepoch:
        for xtrain, ytrain in tepoch: #for every iteration in train datasets
            optimizer.zero_grad()

            xtrain = xtrain.to(device) #both model and input need to be on the same device
            train_prob = model(xtrain) #output of model based
            train_prob = train_prob.cpu() #prevent maxing out gpu memory

            #calculate loss
            loss = loss_fn(train_prob, ytrain) #compare model output to actual output
            loss.backward() #backpropagation
            optimizer.step() #update model parameters (weights)

            # training ends

            train_pred = torch.max(train_prob, 1).indices #converts from probabilities to 1 and 0 (true and false)
            tr_acc += int(torch.sum(train_pred == ytrain)) #adds number of trues

        ep_tr_acc = tr_acc / LEN_TRAIN


    #Evaluate
    model.eval()
    with torch.no_grad(): # don't update gradients
        for xtest, ytest in test_loader:
            xtest = xtest.to(device)
            test_prob = model(xtest)
            test_prob = test_prob.cpu()

            test_pred = torch.max(test_prob, 1).indices
            test_acc += int(torch.sum(test_pred == ytest))

        ep_test_acc = test_acc / LEN_TEST

    end = time()
    duration = (end - start) / 60

    print(f"Epoch: {epoch}, Time: {duration}, Loss: {loss} \nTrain_acc: {ep_tr_acc}, Test_acc: {ep_test_acc}")


100%|████████████████████████████████████████| 31/31 [01:06<00:00,  2.15s/batch]


Epoch: 0, Time: 1.3974175651868184, Loss: 0.0635816752910614 
Train_acc: 0.9394594594594594, Test_acc: 0.7700453857791225


100%|████████████████████████████████████████| 31/31 [01:05<00:00,  2.12s/batch]


Epoch: 1, Time: 1.3914691964785257, Loss: 0.030123181641101837 
Train_acc: 0.9783783783783784, Test_acc: 0.7760968229954615


100%|████████████████████████████████████████| 31/31 [01:05<00:00,  2.11s/batch]


Epoch: 2, Time: 1.3777387539545696, Loss: 0.022034022957086563 
Train_acc: 0.985945945945946, Test_acc: 0.7670196671709532


In [24]:
sample_1 = "/Users/austinjohnson/Downloads/Sample Breast Cancer/9259/aojsdfbajsfd.png" #IDC
sample_2 = "/Users/austinjohnson/Downloads/Non Cancerous/9259_idx5_x1701_y451_class0.png" #NonIDC

img = Image.open(sample_1)
img_tensor = tfm(img)

# print(img_tensor.shape) input size for our model should be 4 dimensions our batches had 4 dimensions

img_tensor = img_tensor[np.newaxis, : ]
img_tensor = img_tensor.to(device)

pred_prob = model(img_tensor)
pred = torch.max(pred_prob, 1).indices
pred = pred.item()

if(pred == 0):
    print("IDC")
else:
    print("Non-IDC")


IDC
